In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import dgl
from dgl.nn import ChebConv


import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg

import os
from tqdm import tqdm
from functools import reduce
import argparse

In [2]:
def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))

def po_linear_model(graph, alpha=1, beta=1, sigma=0.1, gamma=2):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i]  # 

def po_multiplicative_model(graph, alpha=1, sigma=0.1, delta=1, gamma=2): 
    for i in graph.nodes:
        graph.nodes[i]["y"] = ( (alpha + sigma * np.random.normal()) * graph.degree[i]/avg_deg )  *  (1 + delta * graph.nodes[i]["z"] + gamma * sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]]) / len(graph[i]) )


def po_linear_model_square_expo(graph, alpha=1, beta=1, sigma=0.1, gamma=2):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * (sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i])**2   


In [3]:
path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=5)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}

In [4]:
A = np.array(nx.adjacency_matrix(g).todense(), dtype = np.float64)
deg_array = np.array(list(dict(g.degree).values()))

D_inv_A = np.zeros_like(A)

for i in range(num_nodes):
    D_inv_A[i] = A[i] / deg_array[i]

multi_hop_A = torch.load("A_2hop.pkl")


In [5]:
# set diagonal of 2-hop adjacency to 0
for i in range(num_nodes):
    multi_hop_A[i, i] = 0

node_list = list(g.nodes.keys())

def po_2hop_linear_model(graph, z_vec, alpha=1, beta=1, sigma=0.1, gamma=1, r1=1, r2=0.5):        
    y_vec = alpha + beta * z_vec + sigma * np.random.normal(size=(num_nodes, 1)) + gamma * (
    r1 * np.matmul(D_inv_A, z_vec) + r2 * np.matmul(multi_hop_A, z_vec)
)
    for i in range(num_nodes):
        graph.nodes[node_list[i]]["y"] = y_vec[i][0]
        graph.nodes[node_list[i]]["z"] = z_vec[i][0]

In [14]:
# role of merge data

num_repeat = 1000
# ramps = [0.5, 0.5, 0.5, 0.5, 0.5]
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]

In [ ]:

ht_array = np.zeros((num_repeat, len(ramps)))

for seed in tqdm(range(num_repeat)):
    np.random.seed(seed)   

    rollout_index = np.random.uniform(0, 1, size=(num_cluster))   
    
    for num_step in range(len(ramps)):        
        p_list = ramps[num_step:]
        ht_list = []
        for p in p_list:
            # tr_clusters = np.where(np.random.binomial(1, p, size = len(clusters))>0)[0]
            tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
            
            nx.set_node_attributes(g, 0, "z")
            if len(tr_clusters) > 0:
                tr_units = reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters])              
                nx.set_node_attributes(g, {unit:1 for unit in tr_units}, "z")
                
            po_linear_model(g, gamma = 1)
            
            # HT estimator
            mo1, mo0 = 0, 0
            for unit in g.nodes:
                if g.nodes[unit]['z'] == 1:
                    mo1 += g.nodes[unit]['y']
                else:
                    mo0 += g.nodes[unit]['y']
            HT = (mo1/p - mo0/(1-p))/num_nodes
            ht_list.append(HT)
        
        ht_array[seed, num_step] = sum(ht_list)/len(ht_list)
                                    
    torch.save(ht_array, "Result/ht_ramp_cluster.pkl")


 20%|██        | 205/1000 [24:40<1:37:20,  7.35s/it]

# Incremental

In [31]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]

In [32]:

ht_array = np.zeros((num_repeat, len(ramps)))

for seed in tqdm(range(num_repeat)):
    np.random.seed(seed)   

    rollout_index = np.random.uniform(0, 1, size=(num_cluster))   
    
    for num_step in range(len(ramps)):        
        p = ramps[num_step]
        
        # tr_clusters = np.where(np.random.binomial(1, p, size = len(clusters))>0)[0]
        tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
        
        nx.set_node_attributes(g, 0, "z")
        if len(tr_clusters) > 0:
            tr_units = reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters])              
            nx.set_node_attributes(g, {unit:1 for unit in tr_units}, "z")
            
        po_linear_model(g, gamma = 1)
        
        # HT estimator
        mo1, mo0 = 0, 0
        for unit in g.nodes:
            if g.nodes[unit]['z'] == 1:
                mo1 += g.nodes[unit]['y']
            else:
                mo0 += g.nodes[unit]['y']
        HT = (mo1/p - mo0/(1-p))/num_nodes
        
        ht_array[seed, num_step] = HT
                                    
    torch.save(ht_array, "Result/ht_incre_cluster.pkl")


100%|██████████| 1000/1000 [43:38<00:00,  2.62s/it]


# Sqaure Exposure

In [20]:
ramps = [0.5]
num_step = 0

In [27]:
ht_array.mean(axis=0) - 2

array([-0.53721498])

In [26]:
ht_array.mean(axis=0)

array([0.48531315])

In [23]:

ht_array = np.zeros((num_repeat, len(ramps)))

for seed in tqdm(range(num_repeat)):
    np.random.seed(seed)   
    rollout_index = np.random.uniform(0, 1, size=(num_cluster))               
    p = ramps[0]
    ht_list = []
    tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
    
    nx.set_node_attributes(g, 0, "z")
    if len(tr_clusters) > 0:
        tr_units = reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters])              
        nx.set_node_attributes(g, {unit:1 for unit in tr_units}, "z")
        
    po_linear_model_square_expo(g, gamma = 1)
    
    # HT estimator
    mo1, mo0 = 0, 0
    for unit in g.nodes:
        if g.nodes[unit]['z'] == 1:
            mo1 += g.nodes[unit]['y']
        else:
            mo0 += g.nodes[unit]['y']
    HT = (mo1/p - mo0/(1-p))/num_nodes
    ht_list.append(HT)
        
    ht_array[seed, num_step] = sum(ht_list)/len(ht_list)
                                    
    torch.save(ht_array, "Result/Square_ht_cluster.pkl")


100%|██████████| 1000/1000 [08:32<00:00,  1.95it/s]


# Two-hop


In [38]:
ramps = [0.5]
num_step = 0
node_list = list(g.nodes.keys())
node_list = np.array(node_list)
# from node to index in node_list
node_list_index_dict = {}
for node in g.nodes:
    node_list_index_dict[node] = np.where(node_list == node)[0][0]

In [39]:

ht_array = np.zeros((num_repeat, len(ramps)))

for seed in tqdm(range(num_repeat)):
    np.random.seed(seed)   

    rollout_index = np.random.uniform(0, 1, size=(num_cluster))   
    
            
    p = ramps[0]
    z_vector = np.zeros((num_nodes)) 
    tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
    tr_units = list(reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters]))
    for node in tr_units:
        z_vector[node_list_index_dict[node]] = 1
        
    po_2hop_linear_model(g, z_vector.reshape(-1,1)) # add dimension
    
    # HT estimator
    mo1, mo0 = 0, 0
    for unit in g.nodes:
        if g.nodes[unit]['z'] == 1:
            mo1 += g.nodes[unit]['y']
        else:
            mo0 += g.nodes[unit]['y']
    HT = (mo1/p - mo0/(1-p))/num_nodes
    ht_list.append(HT)
        
    ht_array[seed, num_step] = HT
                                    
    torch.save(ht_array, "Result/2hop_ht_cluster.pkl")


100%|██████████| 1000/1000 [02:05<00:00,  7.98it/s]


In [43]:
ht_array.mean() - 2.489

-0.9084201731620909

In [42]:
ht_array.std() 

0.5634622285177334